In [ ]:
import torch
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
import matplotlib.pyplot as plt
import pickle 

import sys
sys.path.append('../common/')
from utils import *
from optimisation import *
from loo_cv import *

In [ ]:
seed=0
torch.manual_seed(seed)

This notebooks computes the test error, the 1d interpolatione error on 3 test cases and the 2d interpolation error in 3 test cases, both in uniform mesh and non-uniform mesh.

In [ ]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [ ]:
class Net(nn.Module):
    def __init__(self,N=10):
            super(Net, self).__init__()
            
            nInputs = int(N*N/2-N/2)
            
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(in_features=nInputs, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Linear(in_features=16, out_features=1)
            )
            # map to positive
            
            
    def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

# Generate features

In [ ]:
def upper_half(A):
    n = A.shape[0]
    return 1/A[np.triu_indices(n, k = 1)]

In [ ]:
# network to be tested:
directory = "plots_inter1d/"
if not os.path.exists(directory):
    os.makedirs(directory)

n=10
path =   '../network/results_sorted_1d_2d/best_model.pt'
model=Net()
model.load_state_dict(torch.load(path))
model.eval()

# Interpolation one dimension with Fallback

In [ ]:
def add_midpoints(x, iterations):
    for itr in range(iterations):
        # Create a new list to store midpoints
        new_x = []
        
        # Iterate through the initial list and calculate midpoints
        for i in range(len(x) - 1):
            new_x.append(x[i])
            midpoint = (x[i] + x[i + 1]) / 2
            new_x.append(midpoint)
        
        # Add the last element of the original list
        new_x.append(x[-1])
        
        # Update the original list with the new list
        x = new_x
        
    return x

In [ ]:
def get_eps_optimisation(x_axis,eps_init):
    closs = 10.
    total_iter = 0
    lr_init_inside = 0.1
    eps, closs, lr_init_inside, iters = optimization_loop_ADAM_fallback(x_axis, eps_init, lr_init_inside)
    total_iter += iters
            
    return eps
                            

In [ ]:
# with fallback
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
import time
import pickle

def one_interpolation(n,x_init,testcase,no_testcase,no_division,theta):
    # create the points based on how many times (no_division) we want to divide the x_init
    x_axis_total=add_midpoints(x_init, no_division)
    replaced=0 
    error=[0 for i in range(5)]
    
    A=np.zeros((n+1,n+1))     
    right_side=np.zeros((n+1,1))    
    w=np.zeros((n+1,1))
    origin=np.array([0,0])
    
    nn_time_accum = 0
    hardy_time_accum = 0
    franke_time_accum = 0
    mfranke_time_accum = 0
    rippa_time_accum = 0
         
    replaced_list = []
    for y in range(2**no_division):
        x_axis1=np.array([x_axis_total[y*(n-1)+idx] for idx in range(n)])   
        x_axis_rippa=x_axis1.reshape(n,1)
        
        x_axis=np.zeros(2*x_axis1.shape[0])
        x_axis[::2] =x_axis1

        x_axis=x_axis.reshape(n,2)
        x_axis = sorted( x_axis, key = lambda x: np.linalg.norm(x - origin ) )#sort inputs
        x_axis = np.reshape(x_axis, (n,2)) 
    
        start = time.process_time()
        X=generate_distance_from_coordinates(x_axis)
        scaled_training_set_flat = upper_half(X).reshape((1,int(n*n/2-n/2)))
        scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)   
        shape_optimisation= model(scaled_training_set_tensor).item()
        nn_time = (time.process_time() - start)

        nn_time_accum += nn_time   
        
        
        for count,shape in enumerate([shape_optimisation]):
            
            for i in range(n):
                for j in range(n):            
                    A[i, j] = phi(shape, x_axis_total[i+y*(n-1)], x_axis_total[j+y*(n-1)])
            A[-1,:] =1.0
            A[:,-1] =1.0
            A[-1,-1]=0.0

            if np.log10(np.linalg.cond(A,'fro')) > theta:#np.isinf(np.log10(np.linalg.cond(A,'fro'))):
                # replace shape parameter
                shape = get_eps_optimisation(x_axis,shape)
                A = get_int_matrix(x_axis,shape)
                #print("replaced")
                replaced+=1
                replaced_list.append([x_axis,shape])
            
            else:
                #print("no replace")
                pass
                
            for i in range(n):
                right_side[i,0]=testcase(x_axis_total[i+y*(n-1)])            
            right_side[-1,0]=0.0 

            # compute the weights
            w=np.linalg.solve(A, right_side) 
        
            x_axis_oversampled = np.zeros((n-1,1)) 
            for p in range(n-1):
                x_axis_oversampled[p]=(x_axis_total[p+y*(n-1)]+x_axis_total[p+1+y*(n-1)])/2.0

            # compute the true values
            true_vals=np.zeros((n-1,1))
            for r in range(9):
                true_vals[r,0]=testcase((x_axis_total[r+y*(n-1)]+x_axis_total[r+1+y*(n-1)])/2.0)

            # compute the approximate values
            s=np.zeros((n-1,n+1))
            for i in range(n):      
                for j in range(n-1):            
                    s[j,i] = phi(shape, (x_axis_total[j+y*(n-1)]+x_axis_total[j+1+y*(n-1)])/2.0, x_axis_total[i+y*(n-1)])
            s[:,-1]=1.0        
            approximator_oversampled=np.matmul(s,w)
       
            for i in range(n-1):
                error[count]=abs(approximator_oversampled[i,0]-true_vals[i,0])**2+error[count]

        
        
    part_error=[math.sqrt(err/len(x_axis_total))  for err in error]
    no_points=len(x_axis_total)
    timings = [nn_time_accum, hardy_time_accum, franke_time_accum, mfranke_time_accum, rippa_time_accum]
    print(f"replaced:{replaced}")
            
    return no_points,part_error,replaced,replaced_list

In [ ]:
testcase1=lambda x: np.exp(np.sin(np.pi*x))
testcase2=lambda x: 1/(1+16*(x**2))
testcase3=lambda x: 1.0 if x>0.5 else 0.0

total_no_division=10
no_testcase=1
n=10

for test_no,testcase in enumerate([testcase1,testcase2,testcase3]):
    no_test=no_testcase
    for theta in [12,16,np.inf]:
        for uniform in ['false']:
            x_vec=[]
            optimisation_error=[]
            hardy_error=[]
            franke_error=[]
            mfranke_error=[]
            rippa_error=[]

            if uniform=='true':
                x_init=[0,1/9,2/9,3/9,4/9,5/9,6/9,7/9,8/9,1]
            elif uniform =='false':
                x_init=[0.5+0.5*np.cos((2*k-1)*np.pi/20.0) for k in range(1,11)]
            x_init.sort()

            for no_division in range(total_no_division):
                        no_points,part_error,nreplaced,replaced_list=one_interpolation(n,x_init,testcase,no_testcase,no_division,theta)
                        x_vec.append(no_points)
                        optimisation_error.append(part_error[0])
                        with open(f'test_{test_no}_{theta}_{uniform}.csv','a') as csvfile:
                            csvfile.write(f"{no_points},{part_error[0]},{nreplaced} \n")
                        pickle.dump(replaced_list,open(f'test_{test_no}_{theta}_{uniform}_{no_division}_list_of_points.pkl','wb'))


            #print(optimisation_error)

            fig=plt.figure()

            plt.loglog(x_vec,optimisation_error,color='r',marker='o')


            plt.xlabel('N',fontsize=12)
            plt.ylabel('Error',fontsize=12)
            plt.savefig(directory+'testcase_fall_'+str(theta)+str(no_testcase)+uniform+'.png')  
            plt.close()
        no_testcase+=1